# Spark Consumer

TODO: 

-find how to make output to 2 places : save json to hdfs and table to hive

-how come sometimes there are 2 tweets in one output, like:
<pre>
+----+--------------------+  
| key|               value|  
+----+--------------------+  
|null|"@SenWarren Need ...|  
|null|"RT @TrinityResis...|  
+----+--------------------+  
</pre>
-how to consume from a kafka topic everything there is up until now (cursor)  
-how to cope with shit like  <pre>\\ud83c\\uddf5\\ud83c\\uddf8\\u2764\\</pre> instead of arabic chars https://stackoverflow.com/questions/18337407/saving-utf-8-texts-in-json-dumps-as-utf8-not-as-u-escape-sequence


-how to dump data to HDFS in large parque files

In [ ]:
from __future__ import print_function
import pyarrow as pa
import sys, os
from time import time, sleep
from itertools import cycle
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode, col, split, from_json, flatten
from pyspark.sql.types import StructType, StringType, MapType, StructField,\
                              BooleanType, DateType, NumericType, IntegerType,\
                              LongType, TimestampType, FloatType, ArrayType


In [ ]:
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-sql-kafka-0-10_2.11:2.4.1 pyspark-shell' 

In [ ]:
#USE WITH CAUTION

#DELETE and recreate the data and checkpoint directories in hdfs 

fs = pa.hdfs.connect(
    host='localhost', 
    port=8020, 
    user='hdfs', 
    kerb_ticket=None, 
    driver='libhdfs', 
    extra_conf=None)

def reset_dir(hdfs_path):
    if fs.exists(hdfs_path):
        fs.delete(hdfs_path,recursive=True)
        print("directory deleted: " + hdfs_path)
    else:
        print("no such dir: " + hdfs_path)

    fs.mkdir(hdfs_path + '/sparkcheckpoint')    
    print("directory created: " + fs.ls(hdfs_path)[0])
    fs.chmod(hdfs_path + '/sparkcheckpoint', 0o777)   
    fs.chmod(hdfs_path, 0o777)    

In [ ]:
def poll_continiously(Query, period = .35, attrname = 'status'):
    #spinner = cycle('←↖↑↗→↘↓↙')
    spinner = cycle('wingardium_leviosa ')
    #spinner = cycle('I solemnly swear that I am up to no good'.replace(' ','_')+' ')
    while True:
        sleep(period)
        val = getattr(Query, attrname)
        print('\r'+next(spinner)+repr(val)[:100]+(' '*100) ,end = '')

In [ ]:
topic = 'TweeterArchive'

In [ ]:
hdfs_archive_path = '/tmp/project/archive'    
hdfs_archive_checkpoint_path = hdfs_archive_path + '/sparkcheckpoint'

hdfs_hive_warehouse = '/user/hive/warehouse'    

hdfs_hive_staging = '/tmp/project/hive/tweets'
hdfs_hive_staging_checkpoint = hdfs_hive_staging + '/sparkcheckpoint'

In [ ]:
reset_dir(hdfs_archive_path)

In [ ]:
reset_dir(hdfs_hive_staging)

In [ ]:
spark.stop()

In [ ]:
spark = SparkSession\
    .builder\
    .appName("StructuredTwitterJsonArchive")\
    .getOrCreate()

In [ ]:
tweet_keys = ['id', 'text','created_at', 'geo', 'coordinates', 'place',
              'quote_count', 'reply_count', 'retweet_count', 'favorite_count', 'user_id' ]

tweet_types = [LongType, StringType, TimestampType, StringType, StringType, StringType, 
               IntegerType, IntegerType, IntegerType, IntegerType, LongType]

# tweet_types  = [StringType]* 11 



user_keys = ['id', 'name', 'screen_name','created_at', 'location', 'url',
             'protected', 'verified', 'followers_count', 'friends_count',
             'listed_count', 'favourites_count', 'statuses_count', 'withheld_in_countries', 'key']

user_types = [LongType, StringType, StringType, TimestampType, StringType, StringType, 
              BooleanType, BooleanType, IntegerType, IntegerType, 
              IntegerType, IntegerType, IntegerType, StringType, LongType]

# user_types = [StringType]* 14


In [ ]:
print(len(tweet_keys))
print(len(tweet_types))
print(len(user_keys))
print(len(user_types))

In [ ]:
def assembleStructType(ST,keys,types):
    ST = ST.add(keys[0],types[0]())
    if len(keys)>1:
        return assembleStructType(ST,keys[1:],types[1:])
    else:
        return ST

test = assembleStructType(StructType(), tweet_keys, tweet_types)
test.fields


In [ ]:
# value schema: { "user": "user_keys", "tweet": "tweet_keys" }

tweet_struct = assembleStructType(StructType(), tweet_keys, tweet_types)

user_struct = assembleStructType(StructType(), user_keys, user_types)

schema = StructType().add("users", user_struct).add("tweets", tweet_struct)

print(schema)

In [ ]:
df = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "localhost:9092") \
  .option("subscribe", topic) \
  .load()\
  .select(col("key").cast("string"), from_json(col("value").cast("string"), schema).alias('value'))


## Raw JSON Data to archive 

In [ ]:
# Json to HDFS sink with partitioning

targetJsonHDFS = df\
    .select(col('key'),col('value.*'))\
    .writeStream\
    .format("json")\
    .outputMode("append")\
    .partitionBy("key")\
    .option("path", "hdfs://localhost:8020" + hdfs_archive_path)\
    .option("checkpointLocation", "hdfs://localhost:8020" + hdfs_archive_checkpoint_path)\
    .start()
#targetJsonHDFS.awaitTermination()

In [ ]:
poll_continiously(targetJsonHDFS)

In [ ]:
targetJsonHDFS.explain()

In [ ]:
targetJsonHDFS.exception()

In [ ]:
targetJsonHDFS.stop()

## Create  HIVE Database

In [ ]:
from pyhive import hive

In [ ]:
hdfs_host = 'localhost'
hdfs_port = 9870
hive_port = 10000
hive_username = 'hdfs'
hive_password = 'naya'
hive_database = 'twitter'
hive_mode = 'CUSTOM'

hive_cnx = hive.Connection(
  host = hdfs_host, 
  port = hive_port, 
  username = hive_username,
  password = hive_password,
  auth = hive_mode)

In [ ]:
# q = f"drop database if exists {hive_database} cascade "

# with hive_cnx.cursor() as cursor:
#     cursor.execute(q)

In [ ]:
q = f"create database if not exists {hive_database}"

with hive_cnx.cursor() as cursor:
    cursor.execute(q)

In [ ]:
hive_cnx = hive.Connection(
  host = hdfs_host, 
  port = hive_port, 
  username = hive_username,
  password = hive_password,
  auth = hive_mode,
  database=hive_database)

## Tweets to HIVE

In [ ]:
table = "tweets"
colNames = tweet_keys 
colTypes = [t().simpleString() for t in tweet_types]


In [ ]:
colCreate = ', '.join([n + ' ' + t for n,t in zip(colNames, colTypes)])
colCreate

In [ ]:
reset_dir(f"{hdfs_hive_warehouse}/{hive_database}.db")

In [ ]:
hdfs_hive_tweets = f"{hdfs_hive_warehouse}/{hive_database}.db/{table}"
hdfs_hive_tweets

In [ ]:
reset_dir(hdfs_hive_tweets)

In [ ]:
q =  f"CREATE EXTERNAL TABLE IF NOT EXISTS {table} ({colCreate}) \
partitioned by (key string)\n \
STORED AS PARQUET \
LOCATION  '{hdfs_hive_tweets}'"

# f"LOAD DATA INPATH '{hdfs_hive_staging}/*.parquet' \
# INTO TABLE tweets \
# partition by key "
print(q)

#TBLPROPERTIES ('avro.schema.url'='hdfs://$tempdir/avroSchema/$tbl.avsc') ;" 

In [ ]:
with hive_cnx.cursor() as cursor:
    print(q)
    cursor.execute(q)

In [ ]:
# Dump new tweets to HDFS as Parquet (small files):
tweets2HIVE = df\
    .select(col('key'),col(f'value.tweets.*'))\
    .writeStream\
    .format("parquet")\
    .outputMode("append")\
    .partitionBy("key")\
    .option("path", "hdfs://localhost:8020" + hdfs_hive_tweets)\
    .option("checkpointLocation", "hdfs://localhost:8020" + hdfs_hive_tweets + "/sparkcheckpoint" )\
    .start()   

In [ ]:
poll_continiously(tweets2HIVE)

In [ ]:
tweets2HIVE.explain()

In [ ]:
tweets2HIVE.exception()

In [ ]:
tweets2HIVE.stop()

## Users to HIVE

In [ ]:
table = "users"
staging = "users_staging"
colNames = user_keys[:-1] + ["last_tweet_at"]
colTypes = [t().simpleString() for t in user_types][:-1] + [StringType().simpleString()]


In [ ]:
colCreate = ', '.join([n + ' ' + t for n,t in zip(colNames, colTypes)])
colCreate

In [ ]:
hdfs_hive_users = f"{hdfs_hive_warehouse}/{hive_database}.db/{table}"
hdfs_hive_users

In [ ]:
reset_dir(hdfs_hive_users)

In [ ]:
hdfs_hive_users_staging = f"{hdfs_hive_warehouse}/{hive_database}.db/{staging}"
hdfs_hive_users_staging

In [ ]:
reset_dir(hdfs_hive_users_staging)

In [ ]:
q1 =  f"CREATE EXTERNAL TABLE IF NOT EXISTS {table} ({colCreate}) \
partitioned by (key bigint) \
STORED AS PARQUET \
LOCATION '{hdfs_hive_users}'"

# f"LOAD DATA INPATH '{hdfs_hive_staging}/*.parquet' \
# INTO TABLE tweets \
# partition by key "
print(q1)

#TBLPROPERTIES ('avro.schema.url'='hdfs://$tempdir/avroSchema/$tbl.avsc') ;" 

In [ ]:
q2 =  f"CREATE EXTERNAL TABLE IF NOT EXISTS {staging} ({colCreate}) \
partitioned by (key bigint) \
STORED AS PARQUET \
LOCATION '{hdfs_hive_users_staging}'"

# f"LOAD DATA INPATH '{hdfs_hive_staging}/*.parquet' \
# INTO TABLE tweets \
# partition by key "
print(q2)

#TBLPROPERTIES ('avro.schema.url'='hdfs://$tempdir/avroSchema/$tbl.avsc') ;" 

In [ ]:
with hive_cnx.cursor() as cursor:
    print(q1)
    print(q2)
    cursor.execute(q1)
    cursor.execute(q2)

In [ ]:
# Dump new users to HDFS as Parquet (small files): .dropDuplicates(subset=['id'])\
users2HIVE = df\
    .select(col(f'value.users.*'),col('key').alias('last_tweet_at'))\
    .writeStream\
    .format("parquet")\
    .outputMode("append")\
    .partitionBy("key")\
    .option("path", "hdfs://localhost:8020" + hdfs_hive_users_staging)\
    .option("checkpointLocation", "hdfs://localhost:8020" + hdfs_hive_users_staging + "/sparkcheckpoint" )\
    .start()   

In [ ]:
poll_continiously(users2HIVE)

In [ ]:
users2HIVE.explain()

In [ ]:
users2HIVE.exception()

In [ ]:
users2HIVE.stop()

## Start HIVE refresher job

In [ ]:


q1 = "insert into users_staging select * from users where key=2010"

q2 ="alter table users drop partition (key=2010)"

q3 = "insert into users partition(key=2010) select * from users_staging"

q4 = "alter table users_staging drop partition (key=2010)"

In [ ]:
#This is probably the right query for the 'merge' of partitions
q5 = """ SELECT id, name, last_tweet_at, followers_count
FROM (
    SELECT id, name, rank() over (partition by id order by followers_count desc) as last_tweet_at, followers_count
    from users_staging
    where key =2010
) ranked_users
WHERE ranked_users.last_tweet_at < 5
ORDER BY id, last_tweet_at """

In [ ]:
with hive_cnx.cursor() as cursor:    
    cursor.execute(q1)      
    cursor.execute(q2)
    cursor.execute(q3)      
    cursor.execute(q4)

In [ ]:
def refresh_hive_table(df, epoch_id):
    hive_cnx = hive.Connection(
        host = 'localhost', 
        port = 10000, 
        username = 'hdfs',
        password = 'naya',
        auth = 'CUSTOM',
        database='twitter')
    with hive_cnx.cursor() as cursor:    
        #cursor.execute(" MSCK REPAIR TABLE tweets ")      
        cursor.execute(" MSCK REPAIR TABLE users_staging ")


In [ ]:
HIVErefresher = df\
                .writeStream\
                .foreachBatch(refresh_hive_table)\
                .trigger( processingTime='1 minute')\
                .start()

In [ ]:
poll_continiously(HIVErefresher)

In [ ]:
HIVErefresher.status

In [ ]:
HIVErefresher.stop()

## Spark.SQL Table

In [ ]:
dbg = df \
    .select(col("value.tweet.*"))\
    .writeStream \
    .queryName("aggregate2") \
    .outputMode("append") \
    .format("memory")\
    .start()

In [ ]:
dbg.stop()

In [ ]:
a = spark.sql("select * from aggregate2") # interactively query in-memory table

In [ ]:
b  = a.write.parquet("hdfs://localhost:8020/tmp/project/tmp.parquet")

In [ ]:
a.show()

In [ ]:
a.show(10,200)


In [ ]:
file = "/tmp/project/hive/key=2019-12-25_18-15/part-00000-c5543d3a-7e1d-467d-a0d7-9b8893fe9bb4.c000.snappy.parquet"
c = spark.read.parquet('hdfs://localhost:8020'+file)

In [ ]:
c.head()

In [ ]:
c.show()

In [ ]:
spark.sql("select * from aggregate2").show(100,1100) # interactively query in-memory table

In [ ]:
len(a)

In [ ]:
a

## Sandbox 
below are cells that I played with to test portions of code


In [ ]:

options={"timestampFormat": "%a %b %d %H:%M:%S %z %Y"} #"yyyy-MM-dd HH:mm:ss"}
options


In [ ]:
df.printSchema()

In [ ]:
 schema = StructType().add("value", StructType().add("user", StringType())
                                            .add("tweet", StringType()))

In [ ]:
schema

In [ ]:
 schema = StructType().add("value", StructType().add("value", StringType()))

In [ ]:
schema

In [ ]:
from pyspark.sql.types import StructType, StringType, MapType, StructField


schema = StructType().add('created_at', StringType(), False).add('id_str', StringType(), False)
schema

In [ ]:
schema = StructType([StructField("value", StringType(), True), True),])

In [ ]:
schema

In [ ]:
schema = StructType([
    StructField("created_at", StringType(), True),
    StructField("id", StringType(), True),
    StructField("text", StringType(), True),
])

In [ ]:
data = spark.read.json(sampleFilePath, schema, multiLine=True)
print(data)

In [ ]:
wat = data.collect()

In [ ]:
wat[0].asDict()

In [ ]:
wat = spark.read.option("multiLine", True).json(sampleFilePath)

In [ ]:
wat.collect()

In [ ]:
jsondf = spark.read.json(Seq(jsonstr).toDS) 

In [ ]:
wat = schema.collect()

In [ ]:
type(wat[0])

In [ ]:
sampleFilePath = "/home/naya/DataEngineerProject/part-00000-7ad71167-a959-4478-929a-6ab7607844d4.c000.json"
sampleFilePath

In [ ]:


with open(sampleFilePath) as f:
    txt = f.read()
    
(txt.__repr__())

json.loads(txt.replace('\\"','\"'))

In [ ]:
sampleFilePath = "hdfs://localhost:8020" +\
                "/tmp/project/archive/key=2019-12-22_10-18/part-00000-7ad71167-a959-4478-929a-6ab7607844d4.c000.json"
sampleFilePath

In [ ]:
from pyspark.sql import SQLContext
sf = sqlContext.read.json(sampleFilePath)

In [ ]:
# json to HDFS sink with trigger

targetJsonHDFS = df\
    .writeStream\
    .format("json")\
    .outputMode("append")\
    .option("path", "hdfs://localhost:8020" + hdfs_archive_path)\
    .trigger(processingTime="5 seconds")\
    .option("checkpointLocation", "hdfs://localhost:8020" + hdfs_archive_checkpoint_path)\
    .start()
targetJsonHDFS.awaitTermination()

In [ ]:
targetJsonHDFS.stop()

In [ ]:
# parquet to HDFS sink example with trigger

targetParquetHDFS = df\
    .writeStream\
    .format("parquet")\
    .outputMode("append")\
    .option("path", "hdfs://localhost:8020/tmp/project")\
    .trigger(processingTime="5 seconds")\
    .option("checkpointLocation", "hdfs://localhost:8020/tmp/sparkcheckpoint/")\
    .start()
targetParquetHDFS.awaitTermination()

In [ ]:
targetParquetHDFS.stop()

In [ ]:
#write parquet files to local linux dir
query = df \
    .writeStream \
    .format("parquet")       \
    .option("path", "/tmp/project")\
    .option("checkpointLocation", "/tmp/sparkcheckpoint/")\
    .outputMode("append")\
    .start()
query.awaitTermination()

In [ ]:
#write parquet files to local linux dir
query = df \
    .writeStream \
    .format("parquet")       \
    .option("path", "/tmp/project")\
    .option("checkpointLocation", "/tmp/sparkcheckpoint/")\
    .outputMode("append")\
    .start()
query.awaitTermination()

In [ ]:
query.stop()

In [ ]:
#output to console
# query = df \
#     .writeStream \
#     .outputMode("append") \
#     .format("console") \
#     .start()
# query.awaitTermination()

In [ ]:
# words = df.select(
#    explode(
#        split(df.value, " ")
#    ).alias("word")
# )

# Generate running word count
#wordCounts = words.groupBy("word").count()

In [ ]:
# query = wordCounts \
#     .writeStream \
#     .outputMode("complete") \
#     .format("console") \
#     .start()
# query.awaitTermination()